In [ ]:
# Rossman Store Sales Prediction

# Steps: 
1. Explatory Data Analysis
2. Time Series Analysis 
    2.1. Predictive Modeling 
3. Results

![](https://m.strelapark.de/fileadmin/_processed_/csm_rossmann_shop_foto_stralsund_1633a5fb67.jpg)

## Used dataset is **rossmann store data**. It operates over 3,000 drug stores in 7 European countries. The challenge is to predict their daily sales for up to six weeks in advance.

In [ ]:
pip install --upgrade pip'

In [ ]:
# Importing required libraries
import numpy as np
import pandas as pd, datetime
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from time import time
import os
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.tsa.arima_model import  ARIMA
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, r2_score
from pandas import DataFrame
import xgboost as xgb
from fbprophet import Prophet
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import datast 
store = pd.read_csv('../input/rossmann-store-sales/store.csv')
train = pd.read_csv('../input/rossmann-store-sales/train.csv', index_col='Date', parse_dates=True)
test = pd.read_csv('../input/rossmann-store-sales/test.csv')
train.shape, test.shape, store.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
store.head()

# **1. Explamatory Data Analysis(EDA)**

### 1.1: Trends & Seasonility 
How the sales vary with month, promo(First promotional Offer), promo2(Second Promotional Offer) and years. 

In [ ]:
train.shape

Train data as almost 1M observations of sales data over the year of appriximatelly (2013-2015). 
Okay, bread Date column in Year, Month, Day, Week columns

In [ ]:
# Extract Year, Month, Day, Wee columns 
train['Year'] = train.index.year
train['Month'] = train.index.month
train['Day'] = train.index.day
train['WeekofYear'] = train.index.weekofyear

train['SalesPerCustomer'] = train['Sales']/train['Customers']

In [ ]:
train.head()

In [ ]:
# Checking the data when the store is closed 
train_store_closed = train[(train.Open == 0)]
train_store_closed.head()

In [ ]:
# Check when the store was closed 
train_store_closed.hist('DayOfWeek')

From this chart, we could see that, 7th day store was mostly clodes. It is Sunday and makes sense. 

In [ ]:
# Check whether there school was closed for holyday 
train_store_closed['SchoolHoliday'].value_counts().plot(kind='bar')

In [ ]:
# Check whether there school was closed for holyday 
train_store_closed['StateHoliday'].value_counts().plot(kind='bar')

Here, The state is closed for (a= Public holyday, b = Easter holyday, c = Christmas and 0 is None)

In [ ]:
# Check the null values
# In here there is no null value 
train.isnull().sum()

In [ ]:
# Number of days with closed stores
train[(train.Open == 0)].shape[0]

In [ ]:
# Okay now check No. of dayes store open but sales zero ( It might be caused by external refurbishmnent)
train[(train.Open == 1) & (train.Sales == 0)].shape[0]

In [ ]:
# Work with store data 
store.head()

In [ ]:
# Check null values 
# Most of the columns has null values 

store.isnull().sum()

In [ ]:
# Replacing missing values for Competiton distance with median
store['CompetitionDistance'].fillna(store['CompetitionDistance'].median(), inplace=True)

In [ ]:
# No info about other columns - so replcae by 0
store.fillna(0, inplace=True)

In [ ]:
# Again check it and now its okay 

store.isnull().sum().sum()

In [ ]:
# Work with test data 
test.head()

In [ ]:
# check null values ( Only one feature Open is empty)
test.isnull().sum()

In [ ]:
# Assuming stores open in test
test.fillna(1, inplace=True)

In [ ]:
# Again check 
test.isnull().sum().sum()

In [ ]:
# Join train and store table 
train_store_joined = pd.merge(train, store, on='Store', how='inner')
train_store_joined.head()

In [ ]:
train_store_joined.groupby('StoreType')['Customers', 'Sales', 'SalesPerCustomer'].sum().sort_values('Sales', ascending='desc')

In [ ]:
# Closed and zero-sales observations 
train_store_joined[(train_store_joined.Open == 0) | (train_store_joined.Sales==0)].shape

So, we have 172,871 observations when the stores were closed or have zero sales.

In [ ]:
# Open & Sales >0 stores
train_store_joined_open = train_store_joined[~((train_store_joined.Open ==0) | (train_store_joined.Sales==0))]
train_store_joined_open

# Correlation Analysis

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(train_store_joined.corr(), annot=True)

#### From the above chart we can see a strong positive correlation between the amount of Sales and Customers visiting the store. We can also observe a positive correlation between a running promotion (Promo = 1) and number of customers.

In [ ]:
# Now plot the sales trend over the month 
sns.factorplot(data = train_store_joined_open, x='Month', y='Sales',
              col ='Promo', hue='Promo2', row='Year')

In [ ]:
# Sales and trend over days
sns.factorplot(data= train_store_joined_open, x='DayOfWeek', y="Sales",
              hue='Promo')

### From the above chart, 0 represents sales and 1 represents promotin in a week. Promotions are not given in weekend (Saturday and Sunday). Because peoples are goinf to buy their household things on the weekend and wothout promotion sales increased in a dramatic way. Promotion are highest on monday and as well as sales are high on that day. 

# **Insights**
### 1. Storetype a has highest customer and sales 
### 2. Storetype b has highest SalesPerCustomer 
### 3. There is no promotion offer in Saturday and Sunday
### 4. Customers are going to buy their goods in tuesday on promotional offer. 

# 2. Time Series Analysis 

In this section we will consider only one store from each store type(a, b, c, d). 

In [ ]:
pd.plotting.register_matplotlib_converters()

Register pandas formatters and converters with matplotlib.

This function modifies the global matplotlib.units.registry dictionary. pandas adds custom converters for

pd.Timestamp

pd.Period

np.datetime64

datetime.datetime

datetime.date

datetime.time

In [ ]:
# Data Preparation: input should be float type 

# our Sales data is int type so lets make it float
train['Sales'] = train['Sales'] * 1.00

train['Sales'].head()

In [ ]:
train.Store.unique()

In [ ]:
# Assigning one store from each category
sales_a = train[train.Store == 2]['Sales']
sales_b = train[train.Store == 85]['Sales'].sort_index(ascending = True) 
sales_c = train[train.Store == 1]['Sales']
sales_d = train[train.Store == 13]['Sales']

frame, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (20, 16))

# Visualize Trend 
sales_a.resample('w').sum().plot(ax = ax1)
sales_b.resample('w').sum().plot(ax = ax2)
sales_c.resample('w').sum().plot(ax = ax3)
sales_d.resample('w').sum().plot(ax = ax4)


# will be used to resample the speed column of our DataFrame
#The 'W' indicates we want to resample by week. At the bottom of this post is a summary of different time frames.
# You could use for Day = d, MOnth = m, Year = y

From the above chart we could see sales of store type A, C has highest sales at the end of the year. December months has christmas season. So, that they get highes salary. At the end of the month their sell decrease. We can not find semiler trend for store B and D, it could be there is no
data for that time perion. Possible reason is "store closed".

# stationarity of Time Seriese

Stationarity means that the statistical properties of a time series do not change over time. Some stationary data is (constant mean, constant variance and constant covariance with time). 

### There are 2 ways to test the stationarity of time series
* A) Rolling Mean: Visualization 
* B) Dicky - Fuller test: Statistical test

**A) Rolling Mean:** A rolling analysis of a time series model is often used to assess the model's stability over time. The window is rolled (slid across the data) on a weekly basis, in which the average is taken on a weekly basis. Rolling Statistics is a visualization test, where we can compare the original data with the rolled data and check if the data is stationary or not.

**B) Dicky -Fuller test:** This test provides us the statistical data such as p-value to understand whether we can reject the null hypothesis. If p-value is less than the critical value (say 0.5), we will reject the null hypothesis and say that data is stationary.

In [ ]:
# lets create a functions to test the stationarity 
def test_stationarity(timeseries):
    # Determine rolling statestics 
    roll_mean = timeseries.rolling(window=7).mean()
    roll_std = timeseries.rolling(window=7).std()
    
    # plotting rolling statestics 
    plt.subplots(figsize = (16, 6))
    orginal = plt.plot(timeseries.resample('w').mean(), color='blue',linewidth= 3, label='Orginal')
    roll_mean = plt.plot(roll_mean.resample('w').mean(), color='red',linewidth= 3, label='Rolling Mean')
    roll_mean = plt.plot(roll_std.resample('w').mean(), color='green',linewidth= 3, label='Rolling Std')
    
    plt.legend(loc='best')
    plt.show()
    
    # Performing Dickey-Fuller test 
    print('Result of Dickey-Fuller test:')
    result= adfuller(timeseries, autolag='AIC')
    
    print('ADF Statestics: %f' %result[0])
    print('P-value: %f' %result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print(key, value)
    

In [ ]:
test_stationarity(sales_a)

In [ ]:
test_stationarity(sales_b)

In [ ]:
test_stationarity(sales_c)

In [ ]:
test_stationarity(sales_d)

# Lets create trends and seasonality 

In [ ]:
# plotting trends and seasonality 

def plot_timeseries(sales,StoreType):

    fig, axes = plt.subplots(2, 1, sharex=True, sharey=False)
    fig.set_figheight(6)
    fig.set_figwidth(20)

    decomposition= seasonal_decompose(sales, model = 'additive',freq=365)

    estimated_trend = decomposition.trend
    estimated_seasonal = decomposition.seasonal
    estimated_residual = decomposition.resid
    
    axes[1].plot(estimated_seasonal, 'g', label='Seasonality')
    axes[1].legend(loc='upper left');
    
    axes[0].plot(estimated_trend, label='Trend')
    axes[0].legend(loc='upper left');

    plt.title('Decomposition Plots')

In [ ]:
plot_timeseries(sales_a, 'a')

In [ ]:
plot_timeseries(sales_b, 'b')

In [ ]:
plot_timeseries(sales_c, 'c')

In [ ]:
plot_timeseries(sales_d, 'd')


From the above plots, we can see that there is seasonality and trend present in our data. So, we'll use forecasting models that take both of these factors into consideration. For example, SARIMAX and Prophet.

# Time Series Forcusting 

## Evaluation Matrics

**1. MAE - Mean Absolute Error:** It is the average of the absolute difference between the predicted values and observed values.
![](https://www.statisticshowto.com/wp-content/uploads/2016/10/MAE.png)

**2. RMSE - Root Mean Square Error:** It is the square root of the average of squared differences between the predicted values and observed values.
![](https://help.innovyze.com/download/attachments/2459040/scadawatch_analytical_function_rmse_formula.png?version=1&modificationDate=1555033531000&api=v2)

# Model 01: Seasonal Autoregressive Integrated Moving Average
In order to use this model, we need to first find out values of **p, d and q. p** represents number of Autoregressive terms - lags of dependent variable.
* q represents number of Moving Average terms
* lagged forecast errors in prediction equation. 
* d represents number of non-seasonal differences.

**To find the values of p, d and q - we use Autocorrelation function (ACF) and Partial Autocorrelation (PACF) plots.**

**ACF** measure of correlation between time series with a lagged version of itself. 
**PACF** measure of correlation between time series with a lagged version of itself but after eliminating the variations already explained by the intervening comparison.

**p value** is the value on x-axis of PACF where the plot crosses the upper Confidence Interval for the first time.

**q value** is the value on x-axis of ACF where the plot crosses the upper Confidence Interval for the first time.


### Autocorrelation function to make ACF and PACF

In [ ]:
def auto_corr(sales):
    lag_acf = acf(sales, nlags=30)
    lag_pacf = pacf(sales,nlags=20,method='ols')
    
    plt.subplot(121)
    plt.plot(lag_acf)
    plt.axhline(y=0, linestyle='--', color='red')
    plt.axhline(y=1.96/np.sqrt(len(sales_a)), linestyle='--', color='red')
    plt.axhline(y=-1.96/np.sqrt(len(sales_a)), linestyle='--', color='red')
    plt.title('ACF')
    
    plt.subplot(122)
    plt.plot(lag_pacf)
    plt.axhline(y=0, linestyle='--', color='red')
    plt.axhline(y=1.96/np.sqrt(len(sales_a)), linestyle='--', color='red')
    plt.axhline(y=-1.96/np.sqrt(len(sales_a)), linestyle='--', color='red')
    plt.title('PACF')


In [ ]:
# ACF and PCF for store A
auto_corr(sales_a)

In [ ]:
# ACF and PCF for store B
auto_corr(sales_b)

In [ ]:
# ACF and PCF for store C
auto_corr(sales_c)

In [ ]:
# ACF and PCF for store D
auto_corr(sales_d)

The above graphs suggest that the p = 2 and q = 2 but let's do a grid search and see which combination of p, q and d gives the lowest Akaike information criterion (**AIC**, which tells us the quality of statistical models for a given set of data. Best model uses the lowest number of features to fit the data.

If we are to predict the sales of each store, we need to consider the whole data set rather than one store of each category. We took one store of each category to understand the tiem series data but from now on, we'll use the whole dataset for modelling

In [ ]:
# Summering sales on per week basis 
# ARIMA = Autoregresive Integrated Moving Average 


train_arima = train.resample('w').mean()
train_arima = train_arima[['Sales']]
train_arima.plot()

In [ ]:
train_arima.head()

### Hyperparamter turing ARIMA model
As discussed above, we have three parameters (p, d and q) for SARIMA model. So, in order to choose the best combination of these parameter, we'll use a grid search. The best combination of parameters will give the lowest AIC score.

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 3
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA: ')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

let's iterate through these combinations to see which one gives the lowest AIC score.

In [ ]:
# Determing p,d,q combinations with AIC scores.
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(train_arima,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

So, we can see that, the above grid search result our optimal paramiter (ARIMA(1, 1, 1)x(1, 1, 1, 12)12 - AIC:1807.3489408440882) 

### Fitting the model

In [ ]:
# Fitting the data to SARIMA model 
model_sarima = sm.tsa.statespace.SARIMAX(train_arima,
                                        order=(1, 1, 1),
                                        seasonal_order=(1,1,1,12),
                                        enforce_stationarity=False,
                                        enforce_invertibility=False)
results_sarima= model_sarima.fit()
print(results_sarima.summary().tables[1])

In [ ]:
# Checking diagnostic plots
results_sarima.plot_diagnostics(figsize=(16, 10))
plt.show()

We can see from the above 'Histogram plus estimated density' plot that our KDE (Kernel Desnity Estimator) plot closely follows the N(0,1) normal distribution plot. The Normal Q-Q plot shows that the ordered distribution of residuals follows the distribution similar to normal distribution. Thus, our model seems to be pretty good.

**Standardized residual plot tells us that there is no major seasonality trend, which is confirmed by Correlogram (autocorrelation) plot. Autocorrelation plot tells us that the time series residuals have low correlation with lagged versions of itself**

In [ ]:
# Model prediction 

pred = results_sarima.get_prediction(start=pd.to_datetime('2015-1-4'), dynamic=False)

# Get confidence interval of forecast 
pred_ci = pred.conf_int()

ax = train_arima['2014':].plot(label='Observed', figsize=(15,7))
pred.predicted_mean.plot(ax=ax, label='One step ahed Forecast', alpha=1)

ax.fill_between(pred_ci.index, 
               pred_ci.iloc[:, 0],
               pred_ci.iloc[:,1],
               color='r', alpha=.1)

ax.set_xlabel('Date')
ax.set_ylabel('Sales')
plt.legend()
plt.show()

train_arima_forecasted = pred.predicted_mean
train_arima_truth = train_arima['2015-01-04':]

rms_arima= sqrt(mean_squared_error(train_arima_truth,train_arima_forecasted))
print('Root Mean Squared Error = ',rms_arima)

In [ ]:
# Save your predicted results for future validation. 
# You could find this results in output sections

train_arima_forecasted.to_csv('predicted_data.csv')
print('Predicted Data Saved in output')

# Model 2: Prophetic 
From our Grid search and foundoptimal parameter we also have another loweset AIC: ARIMA(1, 1, 1)x(0, 1, 1, 12)12 - AIC:1806.29. Lets try to use it 

In [ ]:
# Creating a train dataset
train_prophet = train.copy()
train_prophet.reset_index(level=0, inplace=True)

In [ ]:
# Converting col names to specific names as required by Prophet library
train_prophet = train_prophet.rename(columns = {'Date': 'ds',
                                'Sales': 'y'})
train_prophet.head()

In [ ]:
# Downsampling to week because modelling on daily basis takes a lot of time
ts_week_prophet = train_prophet.set_index("ds").resample("W").sum()
ts_week_prophet.head()

In [ ]:
train_store_joined.columns

# MOdel 2: XGBoost
Now we will drop columns that are correlated (e.g Customers, SalePerCustomer) in addition to merging similar columns into one column (CompetitionOpenSinceMonth, CompetitionOpenSinceYear).

In [ ]:
# Dropping Customers and Sale per customer
ts_xgboost = train_store_joined.copy()
ts_xgboost = ts_xgboost.drop(['Customers', 'SalesPerCustomer', 'PromoInterval'], axis=1)

In [ ]:
ts_xgboost.head()
# Here we do not have any categorical variables so we do not have to convert them into numerical to use in XGBoost 

In [ ]:
# Combining similar columns into one column and dropping old columns
ts_xgboost['CompetitionOpen'] = 12 * (ts_xgboost.Year - ts_xgboost.CompetitionOpenSinceYear) + (ts_xgboost.Month - ts_xgboost.CompetitionOpenSinceMonth)
ts_xgboost['PromoOpen'] = 12 * (ts_xgboost.Year - ts_xgboost.Promo2SinceYear) + (ts_xgboost.WeekofYear - ts_xgboost.Promo2SinceWeek) / 4.0
ts_xgboost = ts_xgboost.drop(["CompetitionOpenSinceMonth", "CompetitionOpenSinceYear"], axis = 1)
ts_xgboost = ts_xgboost.drop(["Promo2SinceWeek", "Promo2SinceYear"], axis = 1)

In [ ]:
# Converting categorical cols to numerical cols and removing old cols
mappings = {0:0, "0": 0, "a": 1, "b": 1, "c": 1}
ts_xgboost["StateHoliday_cat"] = ts_xgboost["StateHoliday"].map(mappings)
ts_xgboost["StoreType_cat"] = ts_xgboost["StoreType"].map(mappings)
ts_xgboost["Assortment_cat"] = ts_xgboost["Assortment"].map(mappings)
ts_xgboost = ts_xgboost.drop(["StateHoliday", "StoreType", "Assortment"], axis = 1)

In [ ]:
# Splitting the data
features = ts_xgboost.drop(["Sales"], axis = 1)
target = ts_xgboost["Sales"]

X_train, X_test, y_train, y_test = model_selection.train_test_split(features, target, test_size = 0.20) 

# Baseline XGBoost 

In [ ]:
# Tuning parameters - using default metrics
params = {'max_depth':6, "booster": "gbtree", 'eta':0.3, 'objective':'reg:linear'} 

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)
watchlist = [(dtrain, 'train'), (dtest, 'eval')]

# Training the model
xgboost = xgb.train(params, dtrain, 100, evals=watchlist,early_stopping_rounds= 100, verbose_eval=True)
         
# Making predictions
preds = xgboost.predict(dtest)

In [ ]:
# RMSE of model
rms_xgboost = sqrt(mean_squared_error(y_test, preds))
print("Root Mean Squared Error for XGBoost:", rms_xgboost)

# Hypertuning XGBoost
Now let's try to decrease the RMSE of XGBoost by passing different values for our hyperparameters in the XGBoost model.

**eta:** It defines the learning rate i.e step size to learn the data in the gradient descent modeling (the basis for XGBoost). The default value is 0.3 but we want to keep the learning rate low to avoid overfitting. So, we'll choose **0.2** as eta

**max_depth:** Maximum depth of a tree. The default value is 6 but we want our model to be more complex and find good predictions. So, let's choose 10 as max depth.

**gamma:** Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger gamma is, the more conservative the algorithm will be. The default value is 0, let's choose a little higher value so as to get good predictions

In [ ]:
# Tuning parameters
params_2 = {'max_depth':10, 'eta':0.1,  'gamma': 2}

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)
watchlist = [(dtrain, 'train'), (dtest, 'eval')]

# Training the model
xgboost_2 = xgb.train(params_2, dtrain, 100, evals=watchlist,early_stopping_rounds= 100, verbose_eval=True)
         
# Making predictions
preds_2 = xgboost_2.predict(dtest)

In [ ]:
# RMSE of model
rms_xgboost_2 = sqrt(mean_squared_error(y_test, preds_2))
print("Root Mean Squared Error for XGBoost:", rms_xgboost_2)

In [ ]:
# Let's see the feature importance
fig, ax = plt.subplots(figsize=(10,10))
xgb.plot_importance(xgboost_2, max_num_features=50, height=0.8, ax=ax)
plt.show()

# Final XGBoost Model:
After hypertuning, we were able to reduce RMSE from 1223.31 to 1176.20 which is great! Now, let's compare the performance of all models

# Results

In [ ]:
# Comparing performance of above three models - through RMSE
rms_arima = format(float(rms_arima))
rms_xgboost_2 = format(float(rms_xgboost_2))

model_errors = pd.DataFrame({
    "Model": ["SARIMA",  "XGBoost"],
    "RMSE": [rms_arima, rms_xgboost_2]
})

model_errors.sort_values(by = "RMSE")

# Model Comparison & Selection
We used the Root Mean Squared Error **(RMSE)** to evaluate and validate the performance of various models used. Let's see which model performed better and why/why not.

a) We can see from the above table that **SARIMA** performs the best than **XGBoost**.

b) It makes sense because **SARIMA is designed specifically for seasonal time series data while XGBoost is a general (though powerful) machine learning approach with various applications.**


Based on the above analysis, we'll choose ARIMA as our final model to predict the sales because it gives us the least RMSE and is well suited to our needs of predicting time series seasonal data.